In [49]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd


class FeaturesLinear(torch.nn.Module):

    def __init__(self, field_dims, output_dim=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias


class FeaturesEmbedding(torch.nn.Module):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)


class FactorizationMachine(torch.nn.Module):

    def __init__(self, reduce_sum=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

In [50]:
class FieldAwareFactorizationMachine(torch.nn.Module):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_dims), embed_dim) for _ in range(self.num_fields)
        ])
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        xs = [self.embeddings[i](x) for i in range(self.num_fields)]
        ix = list()
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                ix.append(xs[j][:, i] * xs[i][:, j])
        ix = torch.stack(ix, dim=1)
        print(ix.shape)
        return ix

In [42]:
dataset.field_dims

array([ 2,  4,  3, 20, 20,  9, 14,  9,  6,  2, 13, 54,  4,  4, 46,  4,  4,
       30,  4, 14, 24, 17], dtype=uint32)

In [44]:
sum(dataset.field_dims)

307

In [41]:
ffm = FieldAwareFactorizationMachine(dataset.field_dims, 4)
ffm.offsets

array([  0,   2,   6,   9,  29,  49,  58,  72,  81,  87,  89, 102, 156,
       160, 164, 210, 214, 218, 248, 252, 266, 290])

In [51]:
class FieldAwareFactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of Field-aware Factorization Machine.
    Reference:
        Y Juan, et al. Field-aware Factorization Machines for CTR Prediction, 2015.
    """

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.linear = FeaturesLinear(field_dims)
        self.ffm = FieldAwareFactorizationMachine(field_dims, embed_dim)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        ffm_term = torch.sum(torch.sum(self.ffm(x), dim=1), dim=1, keepdim=True)
        x = self.linear(x) + ffm_term
        return torch.sigmoid(x.squeeze(1))

In [70]:
import torch
import tqdm
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader

from torchfm.dataset.avazu import AvazuDataset
from torchfm.dataset.criteo import CriteoDataset
from torchfm.dataset.movielens import MovieLens1MDataset, MovieLens20MDataset
from torchfm.model.afi import AutomaticFeatureInteractionModel
from torchfm.model.afm import AttentionalFactorizationMachineModel
from torchfm.model.dcn import DeepCrossNetworkModel
from torchfm.model.dfm import DeepFactorizationMachineModel
from torchfm.model.ffm import FieldAwareFactorizationMachineModel
from torchfm.model.fm import FactorizationMachineModel
from torchfm.model.fnfm import FieldAwareNeuralFactorizationMachineModel
from torchfm.model.fnn import FactorizationSupportedNeuralNetworkModel
from torchfm.model.lr import LogisticRegressionModel
from torchfm.model.ncf import NeuralCollaborativeFiltering
from torchfm.model.nfm import NeuralFactorizationMachineModel
from torchfm.model.pnn import ProductNeuralNetworkModel
from torchfm.model.wd import WideAndDeepModel
from torchfm.model.xdfm import ExtremeDeepFactorizationMachineModel
from torchfm.model.afn import AdaptiveFactorizationNetwork

In [71]:
def get_dataset(name, path):
    if name == 'movielens1M':
        return MovieLens1MDataset(path)
    elif name == 'movielens20M':
        return MovieLens20MDataset(path)
    elif name == 'criteo':
        return CriteoDataset(path)
    elif name == 'avazu':
        return AvazuDataset(path)
    else:
        raise ValueError('unknown dataset name: ' + name)


def get_model(name, dataset):
    """
    Hyperparameters are empirically determined, not opitmized.
    """
    field_dims = dataset.field_dims
    if name == 'lr':
        return LogisticRegressionModel(field_dims)
    elif name == 'fm':
        return FactorizationMachineModel(field_dims, embed_dim=16)
    elif name == 'ffm':
        return FieldAwareFactorizationMachineModel(field_dims, embed_dim=4)
    elif name == 'fnn':
        return FactorizationSupportedNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'wd':
        return WideAndDeepModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'ipnn':
        return ProductNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16,), method='inner', dropout=0.2)
    elif name == 'opnn':
        return ProductNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16,), method='outer', dropout=0.2)
    elif name == 'dcn':
        return DeepCrossNetworkModel(field_dims, embed_dim=16, num_layers=3, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'nfm':
        return NeuralFactorizationMachineModel(field_dims, embed_dim=64, mlp_dims=(64,), dropouts=(0.2, 0.2))
    elif name == 'ncf':
        # only supports MovieLens dataset because for other datasets user/item colums are indistinguishable
        assert isinstance(dataset, MovieLens20MDataset) or isinstance(dataset, MovieLens1MDataset)
        return NeuralCollaborativeFiltering(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2,
                                            user_field_idx=dataset.user_field_idx,
                                            item_field_idx=dataset.item_field_idx)
    elif name == 'fnfm':
        return FieldAwareNeuralFactorizationMachineModel(field_dims, embed_dim=4, mlp_dims=(64,), dropouts=(0.2, 0.2))
    elif name == 'dfm':
        return DeepFactorizationMachineModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'xdfm':
        return ExtremeDeepFactorizationMachineModel(
            field_dims, embed_dim=16, cross_layer_sizes=(16, 16), split_half=False, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'afm':
        return AttentionalFactorizationMachineModel(field_dims, embed_dim=16, attn_size=16, dropouts=(0.2, 0.2))
    elif name == 'afi':
        return AutomaticFeatureInteractionModel(
             field_dims, embed_dim=16, atten_embed_dim=64, num_heads=2, num_layers=3, mlp_dims=(400, 400), dropouts=(0, 0, 0))
    elif name == 'afn':
        print("Model:AFN")
        return AdaptiveFactorizationNetwork(
            field_dims, embed_dim=16, LNN_dim=1500, mlp_dims=(400,400,400), dropouts=(0, 0, 0))
    else:
        raise ValueError('unknown model name: ' + name)


def train(model, optimizer, data_loader, criterion, device, log_interval=1000):
    model.train()
    total_loss = 0
    for i, (fields, target) in enumerate(tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            print('    - loss:', total_loss / log_interval)
            total_loss = 0


def test(model, data_loader, device):
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0):
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())
    return roc_auc_score(targets, predicts)

In [53]:
dataset_path = "../../../../../data4experiments/avazu/train1000.txt"

In [54]:
dataset = get_dataset("avazu", dataset_path)

In [55]:
device = "cpu"
batch_size = 1024
epoch = 50
model_name = "ffm"

In [56]:
dataset.field_dims

array([ 2,  4,  3, 20, 20,  9, 14,  9,  6,  2, 13, 54,  4,  4, 46,  4,  4,
       30,  4, 14, 24, 17], dtype=uint32)

In [57]:
dataset.field_dims.sum()

307

In [58]:
len(dataset.field_dims)

22

In [59]:
df_avazu = pd.read_csv("../../../../../data4experiments/avazu/train1000.txt")
df_avazu.shape

(999, 24)

In [60]:
df_avazu.drop(columns=["id", "click"], inplace=True)

In [61]:
df_avazu.nunique()

hour                  1
C1                    4
banner_pos            2
site_id             122
site_domain         109
site_category        10
app_id               75
app_domain           15
app_category          7
device_id           127
device_ip           909
device_model        341
device_type           4
device_conn_type      3
C14                 105
C15                   3
C16                   3
C17                  68
C18                   4
C19                  27
C20                  54
C21                  21
dtype: int64

In [62]:
train_length = int(len(dataset) * 0.8)
valid_length = int(len(dataset) * 0.1)
test_length = len(dataset) - train_length - valid_length
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    dataset, (train_length, valid_length, test_length))
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=8)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=8)
model = get_model(model_name, dataset).to(device)
criterion = torch.nn.BCELoss()

#torch.save(model, f'{save_dir}/{model_name}.pt')

In [63]:
dataset.NUM_FEATS

22

In [64]:
feat_mapper = dataset._AvazuDataset__get_feat_mapper(dataset_path)
feat_mapper


0it [00:00, ?it/s]
Create avazu dataset cache: counting features: : 999it [00:00, 131859.83it/s]


({1: {'14102100': 0},
  2: {'1002': 0, '1005': 1, '1010': 2},
  3: {'1': 0, '0': 1},
  4: {'d6137915': 0,
   'd9750ee7': 1,
   '5b08c53b': 2,
   '543a539e': 3,
   '85f751fd': 4,
   '43d6df75': 5,
   '6ec06dbd': 6,
   '17caea14': 7,
   '84c7ba46': 8,
   '1fbe01fe': 9,
   '856e6d3f': 10,
   'f9c69707': 11,
   '5ee41ff2': 12,
   'f282ab5a': 13,
   'd7f3460b': 14,
   '6256f5b4': 15,
   '5b4d2eda': 16,
   'e151e245': 17,
   '030440fe': 18},
  5: {'d262cf1e': 0,
   'e16ceb4b': 1,
   '71ed77a0': 2,
   '61eb5bc4': 3,
   '08ba7db9': 4,
   'bb1ef334': 5,
   'f7570339': 6,
   '0dde25ec': 7,
   '16a36ef3': 8,
   '7687a86e': 9,
   '58a89a43': 10,
   '17d996e6': 11,
   '28f93029': 12,
   'f3845767': 13,
   '27e3c518': 14,
   'c7ca3108': 15,
   'c4e18dd6': 16,
   '7e091613': 17,
   '98572c79': 18},
  6: {'f66779e6': 0,
   '50e219e0': 1,
   '335d28a8': 2,
   '76b2941d': 3,
   '3e814130': 4,
   '28905ebd': 5,
   '72722551': 6,
   'f028772b': 7},
  7: {'39947756': 0,
   '54c5d545': 1,
   '5e3f096f': 2,


In [65]:
dataset.field_dims

array([ 2,  4,  3, 20, 20,  9, 14,  9,  6,  2, 13, 54,  4,  4, 46,  4,  4,
       30,  4, 14, 24, 17], dtype=uint32)

In [66]:
np.cumsum(dataset.field_dims)[:-1]

array([  2,   6,   9,  29,  49,  58,  72,  81,  87,  89, 102, 156, 160,
       164, 210, 214, 218, 248, 252, 266, 290], dtype=uint64)

In [67]:
ffm = FieldAwareFactorizationMachine(dataset.field_dims, 4)
ffm.offsets

array([  0,   2,   6,   9,  29,  49,  58,  72,  81,  87,  89, 102, 156,
       160, 164, 210, 214, 218, 248, 252, 266, 290])

In [68]:
ffm.embeddings

ModuleList(
  (0): Embedding(307, 4)
  (1): Embedding(307, 4)
  (2): Embedding(307, 4)
  (3): Embedding(307, 4)
  (4): Embedding(307, 4)
  (5): Embedding(307, 4)
  (6): Embedding(307, 4)
  (7): Embedding(307, 4)
  (8): Embedding(307, 4)
  (9): Embedding(307, 4)
  (10): Embedding(307, 4)
  (11): Embedding(307, 4)
  (12): Embedding(307, 4)
  (13): Embedding(307, 4)
  (14): Embedding(307, 4)
  (15): Embedding(307, 4)
  (16): Embedding(307, 4)
  (17): Embedding(307, 4)
  (18): Embedding(307, 4)
  (19): Embedding(307, 4)
  (20): Embedding(307, 4)
  (21): Embedding(307, 4)
)

In [69]:
optimizer = torch.optim.Adam(params=model.parameters())
for epoch_i in range(epoch):
    train(model, optimizer, train_data_loader, criterion, device)
    auc = test(model, valid_data_loader, device)
    print('epoch:', epoch_i, 'validation: auc:', auc)
auc = test(model, test_data_loader, device)
print('test auc:', auc)


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([799, 231, 4])



100%|██████████| 1/1 [00:01<00:00,  1.41s/it]

100%|██████████| 1/1 [00:00<00:00,  4.95it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([99, 231, 4])
epoch: 0 validation: auc: 0.44364937388193204
torch.Size([799, 231, 4])



100%|██████████| 1/1 [00:01<00:00,  1.35s/it]

100%|██████████| 1/1 [00:00<00:00,  4.84it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

torch.Size([99, 231, 4])
epoch: 48 validation: auc: 0.4964221824686941
torch.Size([799, 231, 4])



100%|██████████| 1/1 [00:01<00:00,  1.41s/it]

100%|██████████| 1/1 [00:00<00:00,  5.29it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([99, 231, 4])
epoch: 49 validation: auc: 0.501788908765653
torch.Size([101, 231, 4])


100%|██████████| 1/1 [00:00<00:00,  5.16it/s]


test auc: 0.4602272727272727


In [89]:
field_dimsfield_dims

NameError: name 'field_dims' is not defined

In [97]:
num_fields = len(dataset.field_dims)
num_fields

22

In [94]:
embeddings = torch.nn.ModuleList([
    torch.nn.Embedding(sum(dataset.field_dims), 4) for _ in range(num_fields)
])

In [98]:
embeddings[0]

Embedding(307, 4)